
# Getting Census Data from CensusReporter
## Marketing Use Case for coffee from Mintel Report - HW for Digital Advertising


In [10]:
import pandas as pd
import requests
from time import sleep
import json

In [7]:
#import msas data
df = pd.read_csv("msas.csv")
geoid = list(df["CBSA"])

geoidstr = []
for i in geoid:
    i = "31000US"+str(i) 
    geoidstr.append(i)

#these are the tables we need. Appending to a list separated only by commas
#as required by the censusreporterapi
tableids = ['B01001', 'B19001', 'B03002']
tableidstring = ','.join(tableids)


In [4]:
#create list we need
geoidlist = []
agelist=[]
incomelist = []
racelist = []

In [11]:
#grab data from CensusReporter
for i in geoidstr:
            requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring, i)  
            loadedjson = requests.get(requesturl)
            parsedjson = json.loads(loadedjson.content.decode('utf-8')) 
            geoidlist.append(parsedjson['geography']) 
            agelist.append(parsedjson['data'][i]['B01001']['estimate']['B01001007'] + 
              parsedjson['data'][i]['B01001']['estimate']['B01001031'] +
              parsedjson['data'][i]['B01001']['estimate']['B01001008'] + 
              parsedjson['data'][i]['B01001']['estimate']['B01001032'] +
              parsedjson['data'][i]['B01001']['estimate']['B01001009'] + 
              parsedjson['data'][i]['B01001']['estimate']['B01001033'] +
              parsedjson['data'][i]['B01001']['estimate']['B01001010'] + 
              parsedjson['data'][i]['B01001']['estimate']['B01001034'] +
              parsedjson['data'][i]['B01001']['estimate']['B01001011'] + 
              parsedjson['data'][i]['B01001']['estimate']['B01001035'] +
              parsedjson['data'][i]['B01001']['estimate']['B01001012'] + 
              parsedjson['data'][i]['B01001']['estimate']['B01001036'])
            incomelist.append(parsedjson['data'][i]['B19001']['estimate']['B19001011'] +
              parsedjson['data'][i]['B19001']['estimate']['B19001012'])
            racelist.append(parsedjson['data'][i]['B03002']['estimate']['B03002012'])
            sleep(0.5) 

In [12]:
#create new list
racelist2 = racelist
geoidlist2 = geoidlist
agelist2 = agelist
incomelist2 = incomelist

racelist3 = []
geoidlist3 = []
agelist3 = []
incomelist3 = []

In [13]:
for n in racelist2:
    if n != None:
        racelist3.append(n)
        geoidlist3.append(geoidlist2[racelist2.index(n)])
        agelist3.append(agelist2[racelist2.index(n)])
        incomelist3.append(incomelist2[racelist2.index(n)])

print("highest count of the audience age18-34:",max(agelist),",",geoidlist[agelist.index(max(agelist))])
print("highest count of the audience income50K-74.9K:",max(incomelist),",",geoidlist[incomelist.index(max(incomelist))])
print("highest count of the audience Hispanic populations:",max(racelist3),",",geoidlist3[racelist3.index(max(racelist3))])


highest count of the audience age18-34: 4806939.0 , {'31000US35620': {'name': 'New York-Newark-Jersey City, NY-NJ-PA Metro Area'}}
highest count of the audience income50K-74.9K: 1038188.0 , {'31000US35620': {'name': 'New York-Newark-Jersey City, NY-NJ-PA Metro Area'}}
highest count of the audience Hispanic populations: 6031492.0 , {'31000US31080': {'name': 'Los Angeles-Long Beach-Anaheim, CA Metro Area'}}


In [14]:
agepercent = []
incomepercent = []
racepercent= []
for i in agelist3:
    agepercent.append(i/max(agelist3))
    
for i in incomelist3:
    incomepercent.append(i/max(incomelist3))
    
for i in racelist3:
    racepercent.append(i/max(racelist3))

dfscore = pd.DataFrame({'geoid':geoidlist3})
dfscore['agepercent'] = agepercent
dfscore['incomepercent'] = incomepercent
dfscore['racepercent'] = racepercent
dfscore['avg_score'] = (dfscore['agepercent'] + dfscore['incomepercent'] + dfscore['racepercent'] ) / 3
dfsorted = dfscore.sort_values(by=['avg_score'],ascending=False)
dfsorted.head(10)

,geoid,agepercent,incomepercent,racepercent,avg_score
384,{'31000US35620': {'name': 'New York-Newark-Jer...,1.000000,1.000000,0.828053,0.942684
361,{'31000US31080': {'name': 'Los Angeles-Long Be...,0.703381,0.682105,1.000000,0.795162
153,{'31000US16980': {'name': 'Chicago-Naperville-...,0.465513,0.562460,0.352293,0.460088
268,{'31000US16980': {'name': 'Chicago-Naperville-...,0.465513,0.562460,0.352293,0.460088
40,{'31000US16980': {'name': 'Chicago-Naperville-...,0.465513,0.562460,0.352293,0.460088
281,{'31000US19100': {'name': 'Dallas-Fort Worth-A...,0.367752,0.456243,0.354150,0.392715
53,{'31000US19100': {'name': 'Dallas-Fort Worth-A...,0.367752,0.456243,0.354150,0.392715
166,{'31000US19100': {'name': 'Dallas-Fort Worth-A...,0.367752,0.456243,0.354150,0.392715
97,{'31000US26420': {'name': 'Houston-The Woodlan...,0.345310,0.396790,0.426063,0.389388
325,{'31000US26420': {'name': 'Houston-The Woodlan...,0.345310,0.396790,0.426063,0.389388


'/Users/pudin/Desktop/digital'